In [1]:
import numpy as np
import pandas as pd

In [2]:
# 读取数据
names = ['mobile','action_time','touchpoint_id','action_rank']
df_valid = pd.read_csv('tp_analysis_base.csv',sep = '\t', names = names)
df_valid['mobile'] = df_valid['mobile'].astype(str)

tid_cat = pd.read_csv('data/id_mapping.csv')

In [4]:
df_zhanbai = df_valid.loc[df_valid.touchpoint_id=='014004000000_tp']\
.groupby('mobile').min()['action_time'].to_frame().reset_index().rename(columns={'action_time':'zhanbai_time'})
df_chengjiao = df_valid.loc[df_valid.touchpoint_id=='011000000000_tp']\
.groupby('mobile').min()['action_time'].to_frame().reset_index().rename(columns={'action_time':'chengjiao_time'})

In [5]:
tid_list = tid_cat.loc[(tid_cat.level_2=='002011000000_tp') | (tid_cat.level_2=='009003000000_tp')].touchpoint_id

In [6]:
df_callsms = df_valid.loc[df_valid.touchpoint_id.isin(tid_list)]

In [7]:
df_callsms = pd.merge(df_callsms,tid_cat[['touchpoint_id','touchpoint_level','level_1','level_2','level_3','level_4']],
         how='left',on='touchpoint_id')

In [9]:
temp = df_callsms[['mobile','level_1','action_time']]\
.append(df_callsms[['mobile','level_2','action_time']].rename(columns={'level_2':'level_1'}))\
.append(df_callsms[['mobile','level_3','action_time']].rename(columns={'level_3':'level_1'}))\
.append(df_callsms[['mobile','level_4','action_time']].rename(columns={'level_4':'level_1'})).rename(columns={'level_1':'touchpoint_id'})

In [10]:
temp = pd.merge(temp,df_zhanbai,how='left',on='mobile')
temp = pd.merge(temp,df_chengjiao,how='left',on='mobile')

In [11]:
def get_first_action(ser):
    if pd.isna(ser['zhanbai_time']) or pd.isna(ser['chengjiao_time']):
        return ser
    elif ser['zhanbai_time']<=ser['chengjiao_time']:
        ser['chengjiao_time']=np.nan
    elif ser['zhanbai_time']>ser['chengjiao_time']:
        ser['zhanbai_time']=np.nan
    return ser

In [12]:
temp = temp.apply(get_first_action,axis=1)

In [13]:
temp = temp.loc[(temp.action_time<temp.zhanbai_time) | (temp.action_time<temp.chengjiao_time)]

#### 不同类型的外呼频次所对应的成交转化率

In [15]:
res = temp.groupby(['mobile','touchpoint_id']).count()[['action_time','zhanbai_time','chengjiao_time']].reset_index()

In [16]:
res['zhanbai_time'] = res.zhanbai_time.replace(0,np.nan)
res['chengjiao_time'] = res.chengjiao_time.replace(0,np.nan)

In [17]:
res = res.groupby(['touchpoint_id','action_time']).count()[['zhanbai_time','chengjiao_time']].reset_index()

In [18]:
temp1 = pd.merge(df_zhanbai,df_chengjiao,how='left',on='mobile')
zhanbai_uv = temp1.loc[pd.isna(temp1.chengjiao_time)|(temp1.zhanbai_time<=temp1.chengjiao_time)].groupby('mobile').count().shape[0]
temp1 = pd.merge(df_chengjiao,df_zhanbai,how='left',on='mobile')
chengjiao_uv = temp1.loc[pd.isna(temp1.zhanbai_time)|(temp1.zhanbai_time>temp1.chengjiao_time)].groupby('mobile').count().shape[0]

In [19]:
temp1 = res.groupby('touchpoint_id').sum()[['zhanbai_time','chengjiao_time']]
res0 = temp1.copy()
res0['zhanbai_time']=zhanbai_uv-temp1.zhanbai_time
res0['chengjiao_time']=chengjiao_uv-temp1.chengjiao_time
res0['action_time']=0
res0 = res0.reset_index()

In [20]:
res = res.append(res0[res.columns])

In [21]:
res['zhanbai_rate'] = res.zhanbai_time/(res.zhanbai_time+res.chengjiao_time)
res['chengjiao_rate'] = res.chengjiao_time/(res.zhanbai_time+res.chengjiao_time)
res['zhanbai_dist'] = res.zhanbai_time/zhanbai_uv
res['chengjiao_dist'] = res.chengjiao_time/chengjiao_uv
res['all_dist'] = (res.zhanbai_time+res.chengjiao_time)/(zhanbai_uv+chengjiao_uv)

In [25]:
res.to_csv('战败成交外呼.csv')

#### 外呼短信次数战败激活率

In [123]:
zhanbai_uv = df_zhanbai.groupby('mobile').count().shape[0]
jihuo_uv = df_jihuo.groupby('mobile').count().shape[0]

In [129]:
temp1 = res.groupby('touchpoint_id').sum()[['zhanbai_time','jihuo_time']].rename(columns={'zhanbai_time':'1','jihuo_time':'2'})

In [132]:
temp1['zhanbai_time'] = zhanbai_uv-temp1['1']
temp1['jihuo_time']=jihuo_uv-temp1['2']

In [134]:
temp1['action_time']=0
temp1['ratio'] = temp1.jihuo_time/temp1.zhanbai_time

In [138]:
temp1 = temp1.reset_index()
res = res.append(temp1[res.columns])

In [140]:
res.to_csv('外呼短信次数转化率.csv')